In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import glob
import cv2
import torch
from torch.utils.data import Dataset,DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision.transforms as transforms
from sklearn.model_selection import StratifiedKFold
import timm
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts, ReduceLROnPlateau, OneCycleLR
from pytorch_lightning import Trainer, seed_everything
import torch.nn.functional as F
from torchmetrics import Accuracy,Precision,Recall,F1Score
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, BackboneFinetuning, EarlyStopping
import wandb
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support

from torch import tensor
from torchmetrics.classification import MultilabelAccuracy
from torch.nn import Sigmoid
from sklearn.utils import shuffle
from torchmetrics.classification import MultilabelF1Score
from sklearn.metrics import f1_score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def create_csv(dataset_path, output_csv_path):
    data = {'path': [], 'class': []}

    for class_folder in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_folder)
        if os.path.isdir(class_path):
            for image_file in os.listdir(class_path):
                if image_file.endswith('.jpg') or image_file.endswith('.png'):
                    image_path = os.path.join(class_path, image_file)
                    data['path'].append(image_path)
                    data['class'].append(class_folder)

    df = pd.DataFrame(data)
    df.to_csv(output_csv_path, index=False)

# Replace these paths with your actual paths
train_dataset_path = '/kaggle/input/resized-orginal-data-set-gbbsl/resized dataset (256,256) BDSL/Train Data'
test_dataset_path = '/kaggle/input/resized-orginal-data-set-gbbsl/resized dataset (256,256) BDSL/Test Data'

train_output_csv = '/kaggle/working/train.csv'
test_output_csv = '/kaggle/working/test.csv'

create_csv(train_dataset_path, train_output_csv)
create_csv(test_dataset_path, test_output_csv)

In [3]:
class Config :
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_df = pd.read_csv('/kaggle/working/train.csv')
    test_df = pd.read_csv('/kaggle/working/test.csv')

In [4]:
train_transform = transforms.Compose([
    transforms.Resize((176, 176)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_transform = transforms.Compose([
    transforms.Resize((176,176)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

In [5]:
import os
import pandas as pd

def create_csv(dataset_path, output_csv_path):
    data = {'path': [], 'class': []}

    for class_folder in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_folder)
        if os.path.isdir(class_path):
            for image_file in os.listdir(class_path):
                if image_file.endswith('.jpg') or image_file.endswith('.png'):
                    image_path = os.path.join(class_path, image_file)
                    # Extract only the numeric part from the class_folder
                    class_numeric = ''.join(filter(str.isdigit, class_folder))
                    data['path'].append(image_path)
                    data['class'].append(class_numeric)

    df = pd.DataFrame(data)
    df.to_csv(output_csv_path, index=False)

# Replace these paths with your actual paths
train_dataset_path = '/kaggle/input/resized-orginal-data-set-gbbsl/resized dataset (256,256) BDSL/Train Data'
test_dataset_path = '/kaggle/input/resized-orginal-data-set-gbbsl/resized dataset (256,256) BDSL/Test Data'

train_output_csv = '/kaggle/working/train.csv'
test_output_csv = '/kaggle/working/test.csv'

create_csv(train_dataset_path, train_output_csv)
create_csv(test_dataset_path, test_output_csv)

In [6]:
train_df = pd.read_csv('/kaggle/working/train.csv')
test_df = pd.read_csv('/kaggle/working/test.csv')

In [7]:
class BDSLdataset(Dataset):
    
    def __init__(self, df, transform=None, unlabelled=False):
        self.df = df
        self.transform = transform
        self.unlabelled = unlabelled
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row.path
        
        if self.unlabelled:
            image = Image.open(img_path)
        else:
            image = Image.open(img_path)
    
        if self.transform:
            tensor_image = self.transform(image)
        
        if self.unlabelled:
            return tensor_image
        else:
            label = row['class']
            tensor_label = torch.tensor(label, dtype=torch.float32)
            
            return tensor_image, tensor_label

In [8]:
train_dataset = BDSLdataset(train_df,transform=train_transform,unlabelled=False)
test_dataset = BDSLdataset(test_df,transform=test_transform,unlabelled=False)

In [9]:
train_loader = DataLoader(
    train_dataset,
    batch_size = 16,
    shuffle = True,
    pin_memory = False,
    drop_last = True,
    num_workers = 2
)

test_loader = DataLoader(
    test_dataset,
    batch_size = 16,
    shuffle = False,
    pin_memory = False,
    num_workers = 2,
    drop_last=True
)

In [10]:
unique_numbers_count = train_df['class'].nunique()
print("Number of unique numbers in the 'class' column:", unique_numbers_count)

Number of unique numbers in the 'class' column: 49


In [11]:
class OliveBackBone(nn.Module):
    
    def __init__(self,num_classes=49,model_name='cspresnet50',pretrained=True):
        super(OliveBackBone,self).__init__()
        self.model =timm.create_model(model_name,pretrained=pretrained)
        self.model.head.fc = nn.Linear(self.model.head.fc.in_features, num_classes)

        
    def forward(self,x):
        x = self.model(x) 
        return x

In [12]:
device = Config.device
model =OliveBackBone().to(device)
optim = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.001)
loss_fn = nn.CrossEntropyLoss()

In [13]:
num_epochs = 1
best_val_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Zero the gradients
        optim.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels.long())  # Assuming labels are LongTensor

        # Backward pass
        loss.backward()

        # Update weights
        optim.step()

        # Update statistics
        train_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    # Calculate average training loss and accuracy
    avg_train_loss = train_loss / len(train_loader)
    train_accuracy = correct_train / total_train

    # Print training epoch statistics
    print(f'Training Epoch [{epoch + 1}/{num_epochs}], '
          f'Loss: {avg_train_loss:.4f}, Accuracy: {train_accuracy:.4f}')
    # Validation loop
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = loss_fn(outputs, labels.long())

            # Update statistics
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    # Calculate average validation loss and accuracy
    avg_val_loss = val_loss / len(test_loader)
    val_accuracy = correct_val / total_val

    # Print validation epoch statistics
    print(f'Validation Epoch [{epoch + 1}/{num_epochs}], '
          f'Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.4f}')

  0%|          | 0/736 [00:00<?, ?it/s]

Training Epoch [1/1], Loss: 0.6864, Accuracy: 0.8169


  0%|          | 0/183 [00:00<?, ?it/s]

Validation Epoch [1/1], Loss: 0.1799, Accuracy: 0.9426
